Import all required packages.

In [ ]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import os
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
import pickle



Category names here stand for folder names.

In [ ]:
Categories=['HealthyMeander','PatientMeander']
print("Type yes to give categories or type no to go with classification of Healthy and Patient");

while(True):
  check=input()
  if(check=='no' or check=='yes'):
    break
  print("Please give a valid input (yes/no)")
if(check=='yes'):
  n=2
  Categories=[Categories]
  print(Categories)
  for i in range(n):
    name= 'HealthyMeander', 'PatientMeander'
    Categories.append(name)
  print(f"If not drive Please upload all the {n} category images in google collab with the same names as given in categories")

Type yes to give categories or type no to go with classification of Healthy and Patient
no


Loading all the images and creating a DataFrame

If you have your images in your google drive, simply mount the google Drive and copy the path of the folder containing all the Category images and Change the datadir variable to that path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Parkinsons: Meander

In [ ]:
flat_data_arr=[]
target_arr=[]
#mount the drive and give path of the parent folder containing all category image folders.
datadir='/content/drive/MyDrive/ParkinsonsNew/MeanderNew'
for i in Categories:
  print(f'loading... category : {i}')
  path=os.path.join(datadir,i)
  for img in os.listdir(path):
    img_array=imread(os.path.join(path,img))
    img_resized=resize(img_array,(150,150,3))
    flat_data_arr.append(img_resized.flatten())
    target_arr.append(Categories.index(i))
  print(f'category loaded:{i} successfully')
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data)
df['Target']=target
df

loading... category : HealthyMeander
category loaded:HealthyMeander successfully
loading... category : PatientMeander
category loaded:PatientMeander successfully


,0,1,2,3,4,5,6,7,8,9,...,67491,67492,67493,67494,67495,67496,67497,67498,67499,Target
0,0.999988,0.999988,0.999989,0.997778,0.997778,0.998864,0.991887,0.991887,0.999113,0.992746,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0
1,0.986793,0.982949,0.999888,0.994250,0.993092,0.999958,0.996286,0.996284,0.996357,0.982221,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0
2,0.981190,0.981190,0.981190,0.993353,0.993353,0.993353,0.984235,0.984235,0.984235,0.974961,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0
3,0.999821,0.982305,0.998542,0.999829,0.977415,0.985757,0.999563,0.983431,0.970476,0.997716,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0
4,0.969472,0.978641,0.974721,0.973953,0.981663,0.977817,0.990375,0.992902,0.991647,0.980946,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,0.901140,0.926955,0.981984,0.894873,0.912070,0.973141,0.910116,0.929631,0.986330,0.899260,...,0.915989,0.923351,0.980046,0.933940,0.953369,0.987054,0.948689,0.964125,0.989791,1
240,0.924395,0.944921,0.986993,0.925985,0.939717,0.985922,0.912830,0.925391,0.985657,0.896980,...,0.953451,0.968492,0.988225,0.956315,0.970297,0.987410,0.946653,0.958819,0.977744,1
241,0.926659,0.932823,0.986137,0.912645,0.920144,0.983016,0.920228,0.924190,0.983477,0.901109,...,0.932482,0.951768,0.981890,0.966337,0.976073,0.994101,0.956109,0.962211,0.986238,1
242,0.935946,0.956511,0.984875,0.947709,0.963529,0.988991,0.929417,0.953525,0.982868,0.942219,...,0.970129,0.985857,0.989818,0.965937,0.982053,0.989516,0.969274,0.984871,0.991312,1


Splitting given data into training and testing segments.

In [ ]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1]
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=77,stratify=y)
print('The data has been split successfully!')

The data has been split successfully!


Training the model, this may take a few minutes.

In [ ]:
param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf','poly']}
svc=svm.SVC(probability=True)
print("The training of the model has started, please wait for a while, this may take few minutes to complete.")
model=GridSearchCV(svc,param_grid)
model.fit(x_train,y_train)
print('The Model has been trained well with the given images.')
model.best_params_

The training of the model has started, please wait for a while, this may take few minutes to complete.
The Model has been trained well with the given images.


{'C': 0.1, 'gamma': 0.0001, 'kernel': 'poly'}

Predicted Array:

In [ ]:
y_pred=model.predict(x_test)
print("The predicted Data is :")
y_pred

The predicted Data is :


array([1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0])

Actual Array:

In [ ]:
print("The actual data is:")
np.array(y_test)

The actual data is:


array([1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0])

In [ ]:
#classification_report(y_pred,y_test)
print(f"The model is {accuracy_score(y_pred,y_test)*100}% accurate")
#confusion_matrix(y_pred,y_test)

The model is 89.79591836734694% accurate


Using Pickle to save the model to the disk.

In [ ]:
pickle.dump(model,open('img_model.p','wb'))
print("Pickle has been dumped successfully!")

NameError: ignored

The Machine-Learning Based Model is created successfully! Now you can test for classification of any image which falls under the mentioned categories.

Testing of model :

In [ ]:
#print(os.path.abspath(os.getcwd()))
model=pickle.load(open('img_model.p','rb'))

url=input('Enter URL of Image')
img=imread(url)
plt.imshow(img)
plt.show()
img_resize=resize(img,(150,150,3))
l=[img_resize.flatten()]
probability=model.predict_proba(l)
for ind,val in enumerate(Categories):
  print(f'{val} = {probability[0][ind]*100}%')
print("The predicted image is : "+Categories[model.predict(l)[0]])
print(f'Is the image a {Categories[model.predict(l)[0]]} ?(y/n)')
while(True):
  b=input()
  if(b=="y" or b=="n"):
    break
  print("please enter either y or n")

if(b=='n'):
  print("What is the image?")
  for i in range(len(Categories)):
    print(f"Enter {i} for {Categories[i]}")
  k=int(input())
  while(k<0 or k>=len(Categories)):
    print(f"Please enter a valid number between 0-{len(Categories)-1}")
    k=int(input())
  print("Please wait for a while for the model to learn from this image :)")
  flat_arr=flat_data_arr.copy()
  tar_arr=target_arr.copy()
  tar_arr.append(k)
  flat_arr.extend(l)
  tar_arr=np.array(tar_arr)
  flat_df=np.array(flat_arr)
  df1=pd.DataFrame(flat_df)
  df1['Target']=tar_arr
  model1=GridSearchCV(svc,param_grid)
  x1=df1.iloc[:,:-1]
  y1=df1.iloc[:,-1]
  x_train1,x_test1,y_train1,y_test1=train_test_split(x1,y1,test_size=0.20,random_state=77,stratify=y1)
  d={}
  for i in model.best_params_:
    d[i]=[model.best_params_[i]]
  model1=GridSearchCV(svc,d)
  model1.fit(x_train1,y_train1)
  y_pred1=model.predict(x_test1)
  print(f"The model is now {accuracy_score(y_pred1,y_test1)*100}% accurate")
  pickle.dump(model1,open('img_model.p','wb'))
print("Thank you for your feedback")


NameError: ignored